In [ ]:
import os
import numpy as np
import pandas as pd
from skimage import draw
import pydicom
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
def load_ct_images(ct_folder):
    """Load CT DICOM images, convert to HU, and return a 3D numpy array."""
    slices = []
    for filename in sorted(os.listdir(ct_folder)):
        if filename.endswith('.dcm'):
            dicom = pydicom.dcmread(os.path.join(ct_folder, filename))
            slices.append(dicom)
    
    # Sort slices based on the Image Position Patient to get correct slice order
    slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
    
    # Get pixel data and scaling factors
    image_data = np.stack([s.pixel_array for s in slices], axis=-1)
    
    # Convert to Hounsfield Units (HU)
    rescale_slope = slices[0].RescaleSlope if 'RescaleSlope' in slices[0] else 1
    rescale_intercept = slices[0].RescaleIntercept if 'RescaleIntercept' in slices[0] else 0

    hu_image = image_data * rescale_slope + rescale_intercept

    # Get pixel spacing and origin
    pixel_spacing = list(slices[0].PixelSpacing)  # Convert MultiValue to list
    slice_thickness = float(slices[0].SliceThickness)  # Ensure this is a float
    pixel_spacing.append(slice_thickness)  # Now it's safe to append
    
    origin = slices[0].ImagePositionPatient  # Origin of the first slice in mm
    
    return hu_image, pixel_spacing, origin

def load_rtstruct(rtstruct_path):
    """Load RTSTRUCT file and return contour data for 'Heart'."""
    rtstruct = pydicom.dcmread(rtstruct_path)
    
    for roi in rtstruct.StructureSetROISequence:
        # if roi.ROIName == 'Heart':
        if roi.ROIName == 'GTV-1':
            roi_number = roi.ROINumber
            break
    
    for contour in rtstruct.ROIContourSequence:
        if contour.ReferencedROINumber == roi_number:
            heart_contours = contour.ContourSequence
            break
    return heart_contours

def get_mask_from_contours(contours, image_shape, pixel_spacing, origin):
    """Convert contours into a binary mask."""
    mask = np.zeros(image_shape, dtype=np.uint8)
    
    # Loop through each contour in the RTSTRUCT file
    for contour in contours:
        # Contour points are in physical space (mm), we need to convert them to voxel indices
        contour_points = np.array(contour.ContourData).reshape(-1, 3)

        # Convert contour points from DICOM physical coordinates (mm) to voxel coordinates
        x_coords = (contour_points[:, 0] - origin[0]) / pixel_spacing[0]
        y_coords = (contour_points[:, 1] - origin[1]) / pixel_spacing[1]
        
        # Find the z-index (slice index) by comparing z position to the slice positions
        z_pos = contour_points[0, 2]  # Z-coordinate (all points in one contour are in one slice)
        z_index = int(round((z_pos - origin[2]) / pixel_spacing[2]))

        # Ensure z_index is within bounds
        if z_index < 0 or z_index >= image_shape[2]:
            continue

        # Get integer pixel coordinates from physical coordinates
        rr, cc = draw.polygon(y_coords, x_coords, shape=image_shape[:2])

        # Apply the mask for this specific slice
        mask[rr, cc, z_index] = 1

    return mask


In [ ]:
# Example usage

dcm_folder = Path.cwd().parent / "src" / "test" / "Dicom" / "IBSI1_CT_phantom"
ct_folder = dcm_folder / "CT_00000"
print(ct_folder)

rtstruct_path = dcm_folder / "RTst_00000" / "DCM_RS_00060.dcm"
print(rtstruct_path)

ct_image, pixel_spacing, origin = load_ct_images(ct_folder)
heart_contours = load_rtstruct(rtstruct_path)

# Create mask for heart
heart_mask = get_mask_from_contours(heart_contours, ct_image.shape, pixel_spacing, origin)

# print(type(heart_mask), heart_mask.shape)
# print(np.where(heart_mask==1))
plt.imshow(heart_mask[:,:,37])

In [ ]:
def dfs(matrix, x, y, visited, cluster):
    # Directions for movement: right, down, left, up, and the four diagonal directions
    directions = [
        (0, 1),   # right
        (1, 0),   # down
        (0, -1),  # left
        (-1, 0),  # up
        (1, 1),   # down-right
        (1, -1),  # down-left
        (-1, 1),  # up-right
        (-1, -1)  # up-left
    ]
    
    visited[x][y] = True
    cluster.append((x, y))

    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy
        
        # Check boundaries and whether the new cell is part of the cluster
        if (0 <= new_x < matrix.shape[0] and
            0 <= new_y < matrix.shape[1] and
            not visited[new_x][new_y] and
            matrix[new_x][new_y] > 129):
            dfs(matrix, new_x, new_y, visited, cluster)

def find_clusters(matrix, threshold):
    visited = np.zeros(matrix.shape, dtype=bool)
    clusters = []
    
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if matrix[i][j] > threshold and not visited[i][j]: 
                cluster = []
                dfs(matrix, i, j, visited, cluster)
                if len(cluster) >= 4: 
                    clusters.append(cluster)
    
    return clusters


In [ ]:
def agatson_score(filtered_array, cluster, pixel_spacing):
    # Print the values of filtered_array at the coordinates in cluster
    score = 0
    volume_score = 0
    maximum = 0
    # print(pixel_spacing)
    for x, y in cluster:
        try:
            # Access the value in filtered_array at the given coordinates
            value = filtered_array[x, y]
            if maximum < value:
                maximum = value
            # score += (0.9765625*0.9765625)    
            score += (pixel_spacing[0]*pixel_spacing[1])    
            # volume_score += (0.9765625*0.9765625*5)                                       
            volume_score += (pixel_spacing[0]*pixel_spacing[1]*pixel_spacing[2])                                       
        except IndexError:
            print(f"Coordinates ({x}, {y}) are out of bounds for the array with shape {filtered_array.shape}.")
    if 130 <= maximum <= 199:
        final_score = score
    if 199 < maximum <= 299:
        final_score = 2*score
    if 299 < maximum <= 399:
        final_score = 3*score
    if 399 < maximum:
        final_score = 4*score
    # print(score)            
    return final_score, volume_score    

In [ ]:

total_score = 0
total_volume_score = 0    
if os.path.exists(rtstruct_path) and os.path.exists(ct_folder):
    ct_image, pixel_spacing, origin = load_ct_images(ct_folder)
    heart_contours = load_rtstruct(rtstruct_path)
    heart_mask = get_mask_from_contours(heart_contours, ct_image.shape, pixel_spacing, origin)      
    for i in np.arange(np.shape(heart_mask)[2]):
        masked_ct_slice = np.where(heart_mask[:,:,i] > 0, ct_image[:,:,i], np.nan)
        rows_to_keep = ~np.all(np.isnan(masked_ct_slice), axis=1)

        # Step 2: Create a mask for columns that are not all NaN
        cols_to_keep = ~np.all(np.isnan(masked_ct_slice), axis=0)

        # Step 3: Filter the original array to keep only the rows and columns that are not all NaN
        filtered_array = masked_ct_slice[rows_to_keep][:, cols_to_keep]
        matrix = filtered_array

        # Find clusters with at least 4 adjacent squares containing values > 129
        clusters = find_clusters(matrix, threshold=129)

        for cluster in clusters:
            if len(cluster) >0:
                total_score+=agatson_score(filtered_array,cluster, pixel_spacing)[0]
                total_volume_score += agatson_score(filtered_array,cluster, pixel_spacing)[1]
    print({'PatientID': 'IBSI1_CT_phantom', 'Agatson_score': total_score, 'Volume_score': total_volume_score})  
else:
    print({'PatientID': 'IBSI1_CT_phantom', 'Agatson_score': 'retry', 'Volume_score': 'retry'})              